In [92]:
import pandas as pd
import mysql.connector

In [93]:
# MySQL 연결 설정
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="0000",
    database="TravelDB"
)

In [94]:
#!pip install sqlalchemy mysql-connector-python

In [95]:
# 여행지 및 숙박지 데이터 불러오기
query_travel = "SELECT * FROM travel_locations;"
# query_hotel = "SELECT * FROM accommodations;"
df_travel = pd.read_sql(query_travel, conn)
df_hotel = pd.read_csv("accommodations.csv")
# 사용자-숙박지 평점 데이터 불러오기
query_rating = "SELECT user_id, location_id, rating FROM user_ratings;"
df_ratings = pd.read_sql(query_rating, conn)
conn.close()

C:\Users\admin\AppData\Local\Temp\ipykernel_6532\3608830782.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_travel = pd.read_sql(query_travel, conn)
C:\Users\admin\AppData\Local\Temp\ipykernel_6532\3608830782.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ratings = pd.read_sql(query_rating, conn)


In [96]:
df_travel.head(2)

,id,name,category,description,latitude,longitude,popularity_score,user_id
0,1,여행지_6043,문화유산,이곳은 아름다운 문화유산 여행지입니다.,35.265994,124.875155,5,41
1,2,여행지_9010,해변,이곳은 아름다운 문화유산 여행지입니다.,34.462090,128.244010,10,19


In [97]:
df_hotel.head()

,accommodation_id,accommodation_name,accommodation_type,location_id
0,1,해변 리조트,리조트,1
1,2,산장,로지,2
2,3,중앙 호텔,호텔,3
3,4,공원 옆 여관,여관,4
4,5,역사적 호텔,호텔,5


In [98]:
df_ratings.head(2)

,user_id,location_id,rating
0,11,1,3
1,50,2,3


In [99]:
df_travel.name.value_counts()

name
여행지_5521    2
여행지_104     2
여행지_9196    2
여행지_1628    2
여행지_8427    2
           ..
여행지_3248    1
여행지_2515    1
여행지_8345    1
여행지_8349    1
여행지_9373    1
Name: count, Length: 350, dtype: int64

### 여행지 추천을 위한 콘텐츠 기반 필터링
- 여행지의 tags 또는 category 정보를 TF-IDF 벡터화하여 유사한 장소를 추천합니다.

In [100]:
# (TF-IDF 벡터화)
# TF-IDF (Term Frequency-Inverse Document Frequency)
# 텍스트 데이터를 숫자로 변환
# 주어진 문장에서 중요한 단어일수록 높은 가중치를 부여
# 단순 단어 빈도수(TF)만 고려하는 것이 아니라, 문서 전체에서 
# 얼마나 희귀한 단어인지(IDF)까지 반영하여 특징을 추출
from sklearn.feature_extraction.text import TfidfVectorizer

# cosine_similarity (코사인 유사도)
# 벡터화된 문장들 간의 **유사도(Similarity)**를 계산
# 두 개의 문장을 비교할 때, 각 단어의 빈도수나 중요도를 고려하여 
# 두 벡터가 얼마나 유사한 방향을 갖는지를 측정
from sklearn.metrics.pairwise import cosine_similarity

In [101]:
# 여행지의 태그 또는 설명을 벡터화
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_travel['description']) 

In [102]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [103]:
# 특정 여행지와 유사한 장소 추천 함수
def recommend_travel(destination_name, top_n=5):
    idx = df_travel[df_travel['name'] == destination_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    
    travel_indices = [i[0] for i in sim_scores]
    return df_travel.iloc[travel_indices][['name', 'category']]

In [132]:
recommend_travel("여행지_6043")

,name,category
1,여행지_9010,해변
6,여행지_7532,자연
11,여행지_4758,자연
15,여행지_811,해변
20,여행지_2812,문화유산


### 협업 필터링을 통한 숙박지 추천
- 사용자별 숙박지 평점을 기반으로 Surprise 라이브러리를 사용하여 추천 모델을 만듭니다.

In [105]:
#!pip install scikit-surprise

In [114]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

In [115]:
df_ratings.head(2)

,user_id,location_id,rating
0,11,1,3
1,50,2,3


In [116]:
# Surprise용 데이터 변환
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'location_id', 'rating']], reader)

In [133]:
# SVD 알고리즘 적용
model = SVD()
cross_validate(model, data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2646  1.1376  1.3592  1.3663  1.2994  1.2854  0.0830  
MAE (testset)     0.9985  0.8741  1.0575  1.0434  1.0282  1.0003  0.0661  
Fit time          0.01    0.00    0.00    0.02    0.00    0.01    0.01    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.26461334, 1.1376062 , 1.35923743, 1.36628909, 1.29941689]),
 'test_mae': array([0.99851993, 0.87412508, 1.05747235, 1.04338194, 1.02819815]),
 'fit_time': (0.006969928741455078,
  0.0041904449462890625,
  0.0,
  0.01572728157043457,
  0.0),
 'test_time': (0.0009970664978027344, 0.0, 0.0, 0.0, 0.0)}

In [134]:
# 특정 사용자의 숙박지 추천
trainset = data.build_full_trainset()
model.fit(trainset)

In [135]:
def recommend_hotel(user_id, top_n=5):
    accommodation_ids = df_hotel['accommodation_id'].unique()
    predictions = [model.predict(user_id, aid) for aid in accommodation_ids]
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    top_accommodations = [pred.iid for pred in predictions[:top_n]]
    return df_hotel[df_hotel['accommodation_id'].isin(top_accommodations)][['accommodation_name', 'location_id']]

In [137]:
# 테스트
recommend_hotel(32)

,accommodation_name,location_id
1,산장,2
3,공원 옆 여관,4
5,사파리 캠프,6
7,사원 휴양지,8
10,정원 빌라,11
